# FastTrak Start Pilot Program Limited English Proficiency Population Analysis 

Using American Community Survey 5-Year Estimates, map top Non-English Languages spoken in
Low-Income Census Tracts in the I-880 corridor. Additionally, provide tabular data including total tract population, and total low-income population. Provide total population of people that speak English 'Less than Very Well' and the share a given population by languages spoken at home for every tract within the I-880 corridor. 

In [5]:
import sys
import getpass
import requests 
import json
import os

user = getpass.getuser()
sys.dont_write_bytecode = True

# for DataViz team members
sys.path.insert(0, '/Users/{}/Box/Utility Code'.format(user))

from utils_io import *

#Geoimports
import geopandas as gp

## Query American Community Survey API

#### [Census American Community Survey 5-Year Data API Documentation](https://www.census.gov/data/developers/data-sets/acs-5year.html)

In [6]:
census_api_key = os.environ.get('CENSUS_API_KEY')

In [11]:
acs_table_vars = pd.read_csv('data/acs_table_variables_income_language.csv',
                             dtype={'Table_Id':'str','Line_Num':'str'})

In [12]:
acs_table_vars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Table_ID                  41 non-null     object
 1   Line_Num                  41 non-null     object
 2   ACS_Variable_Definitnion  41 non-null     object
 3   ACS_Table_Definition      41 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [13]:
#Fill leading zeros for line number column to prepare for concatenation 
acs_table_vars['Line_Num'] = acs_table_vars['Line_Num'].apply(lambda x: x.zfill(3))

In [14]:
#Concat table id, line number, and add 'E' for estimate
acs_table_vars['ACS_Table_Variable'] = (acs_table_vars['Table_ID'].str.strip() + 
                                        '_' + 
                                        acs_table_vars['Line_Num'] + 'E')

In [15]:
acs_table_vars.head(5)

,Table_ID,Line_Num,ACS_Variable_Definitnion,ACS_Table_Definition,ACS_Table_Variable
0,B01003,001,Universe: Total population,Total Population,B01003_001E
1,C17002,001,Universe: Total population for whom poverty st...,Ratio of Income to Poverty Level in the Past 1...,C17002_001E
2,C17002,008,2.00 and over,Ratio of Income to Poverty Level in the Past 1...,C17002_008E
3,C16001,001,Universe: Population 5 years and over,Language Spoken at Home for the Population 5 Y...,C16001_001E
4,C16001,002,Speak only English,Language Spoken at Home for the Population 5 Y...,C16001_002E


In [16]:
acs_vars_lst = acs_table_vars['ACS_Table_Variable'].tolist()

In [19]:
var = ','.join(acs_vars_lst)
counties = '001,013,041,055,075,081,085,095,097'
state = '06'
base_url = 'https://api.census.gov/data/2018/acs/acs5?'

query_params = {
    'get': var,
    'for': 'tract:*',
    'in': [
        f'county:{counties}',
        f'state:{state}',
    ],
    'key': census_api_key
}
rq = requests.get(base_url, params=query_params)
data = rq.json()
acs_df = pd.DataFrame(data[1:],columns=data[0])

In [174]:
# Convert columns to numeric
acs_df[acs_vars_lst] = acs_df[acs_vars_lst].apply(pd.to_numeric)

In [175]:
acs_df.columns

Index(['B01003_001E', 'C17002_001E', 'C17002_008E', 'C16001_001E',
       'C16001_002E', 'C16001_003E', 'C16001_004E', 'C16001_005E',
       'C16001_006E', 'C16001_007E', 'C16001_008E', 'C16001_009E',
       'C16001_010E', 'C16001_011E', 'C16001_012E', 'C16001_013E',
       'C16001_014E', 'C16001_015E', 'C16001_016E', 'C16001_017E',
       'C16001_018E', 'C16001_019E', 'C16001_020E', 'C16001_021E',
       'C16001_022E', 'C16001_023E', 'C16001_024E', 'C16001_025E',
       'C16001_026E', 'C16001_027E', 'C16001_028E', 'C16001_029E',
       'C16001_030E', 'C16001_031E', 'C16001_032E', 'C16001_033E',
       'C16001_034E', 'C16001_035E', 'C16001_036E', 'C16001_037E',
       'C16001_038E', 'state', 'county', 'tract'],
      dtype='object')

In [176]:
#Rename columns to human-readable
cols = {'B01003_001E': 'total_population',
        'C17002_001E': 'total_pop_pov',
        'C17002_008E': 'pop_200_pct_and_over_fpl',
        'C16001_001E': 'pop_5_years_over',
        'C16001_002E': 'pop_english_only',
        'C16001_004E': 'spanish_english_vw',
        'C16001_005E': 'spanish_english_lvw',
        'C16001_007E': 'french_english_vw',
        'C16001_008E': 'french_english_lvw',
        'C16001_010E': 'german_english_vw',
        'C16001_011E': 'german_english_lvw',
        'C16001_013E': 'russian_english_vw',
        'C16001_014E': 'russian_english_lvw',
        'C16001_016E': 'other_indo_english_vw',
        'C16001_017E': 'other_indo_english_lvw',
        'C16001_019E': 'korean_english_vw',
        'C16001_020E': 'korean_english_lvw',
        'C16001_022E': 'chinese_english_vw',
        'C16001_023E': 'chinese_english_lvw',
        'C16001_025E': 'vietnamese_english_vw',
        'C16001_026E': 'vietnamese_english_lvw',
        'C16001_028E': 'tagalog_english_vw',
        'C16001_029E': 'tagalog_english_lvw',
        'C16001_031E': 'other_asian_english_vw',
        'C16001_032E': 'other_asian_english_lvw',
        'C16001_034E': 'arabic_english_vw',
        'C16001_035E': 'arabic_english_lvw',
        'C16001_037E': 'other_unspec_english_vw',
        'C16001_038E': 'other_unspec_english_lvw'
}
acs_df.rename(columns=cols,inplace=True)

In [177]:
acs_df.head(5)

,total_population,total_pop_pov,pop_200_pct_and_over_fpl,pop_5_years_over,pop_english_only,C16001_003E,spanish_english_vw,spanish_english_lvw,C16001_006E,french_english_vw,...,other_asian_english_lvw,C16001_033E,arabic_english_vw,arabic_english_lvw,C16001_036E,other_unspec_english_vw,other_unspec_english_lvw,state,county,tract
0,3825,3702,3016,3624,3009,329,181,148,82,82,...,17,0,0,0,0,0,0,06,097,153403
1,10481,10450,8579,10035,6894,2624,1618,1006,0,0,...,0,116,106,10,16,16,0,06,097,153801
2,2456,2456,2195,2362,2034,280,100,180,10,10,...,0,0,0,0,0,0,0,06,097,150100
3,7481,7392,6255,7105,5272,1578,903,675,0,0,...,0,0,0,0,0,0,0,06,097,150603
4,5021,5021,4326,4711,4154,240,49,191,34,25,...,0,23,23,0,17,10,7,06,097,150702


In [178]:
#add geoid column
acs_df['GEOID'] = acs_df['state'] + acs_df['county'] + acs_df['tract']

In [179]:
#subset columns
sub_cols= [
    'GEOID',
    'total_population',
    'total_pop_pov',
    'pop_200_pct_and_over_fpl',
    'pop_5_years_over',
    'pop_english_only',
    'spanish_english_vw',
    'spanish_english_lvw',
    'french_english_vw',
    'french_english_lvw',
    'german_english_vw',
    'german_english_lvw',
    'russian_english_vw',
    'russian_english_lvw',
    'other_indo_english_vw',
    'other_indo_english_lvw',
    'korean_english_vw',
    'korean_english_lvw',
    'chinese_english_vw',
    'chinese_english_lvw',
    'vietnamese_english_vw',
    'vietnamese_english_lvw',
    'tagalog_english_vw',
    'tagalog_english_lvw',
    'other_asian_english_vw',
    'other_asian_english_lvw',
    'arabic_english_vw',
    'arabic_english_lvw',
    'other_unspec_english_vw',
    'other_unspec_english_lvw'
]
acs_df = acs_df[sub_cols].copy()

## Flag low-income tracts

1. Calculate low-income population 
2. Calculate share of low-income population
3. Flag tracts .5 standard deviation from mean

This methodology follows the Communities of Concern methdology
[Communities of Concern Methodology](https://github.com/BayAreaMetro/Spatial-Analysis-Mapping-Projects/tree/master/Project-Documentation/Communities-of-Concern)

In [180]:
#1. Calculate low income population
acs_df['pop_below_200_pct_fpl'] = (acs_df['total_pop_pov'] - 
                          acs_df['pop_200_pct_and_over_fpl'])

In [181]:
#2. Calculate share of low income population 
acs_df['share_below_200_pct_fpl'] = np.where(acs_df['total_pop_pov'] == 0,0,
                                 (acs_df['pop_below_200_pct_fpl'] / 
                                  acs_df['total_pop_pov']))

In [182]:
#3. Flag tracts .5 standard deviation from mean
cond = ((acs_df['share_below_200_pct_fpl'].mean().round(decimals=2)) + 
        ((.5 * acs_df['share_below_200_pct_fpl'].std().round(decimals=2)).round(decimals=2)))
        
acs_df['low_income'] = np.where(acs_df['share_below_200_pct_fpl'] > cond, 1,0)

In [183]:
acs_df[acs_df['low_income'] == 1]

,GEOID,total_population,total_pop_pov,pop_200_pct_and_over_fpl,pop_5_years_over,pop_english_only,spanish_english_vw,spanish_english_lvw,french_english_vw,french_english_lvw,...,tagalog_english_lvw,other_asian_english_vw,other_asian_english_lvw,arabic_english_vw,arabic_english_lvw,other_unspec_english_vw,other_unspec_english_lvw,pop_below_200_pct_fpl,share_below_200_pct_fpl,low_income
6,06097151301,4874,4874,3439,4432,2520,766,726,9,0,...,0,0,0,0,0,11,0,1435,0.294419,1
7,06097152203,3501,3487,2417,3406,2996,166,42,23,0,...,0,0,0,0,0,18,63,1070,0.306854,1
9,06097153200,8621,8614,5135,7758,4166,1186,1764,30,0,...,72,31,151,0,0,42,0,3479,0.403877,1
18,06001401500,2663,2663,1520,2579,2058,138,134,19,0,...,19,24,0,10,10,68,26,1143,0.429215,1
20,06001402200,2527,2515,1240,2409,1570,177,253,0,0,...,9,31,0,0,0,0,0,1275,0.506958,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,06001410300,3851,3851,1994,3505,1035,1050,1273,8,5,...,0,50,46,0,0,21,0,1857,0.482212,1
1568,06001423200,3004,2981,1773,2812,1909,333,265,7,0,...,0,63,6,0,33,0,0,1208,0.405233,1
1569,06001423300,3764,3764,2691,3504,2919,161,220,18,0,...,0,37,48,0,0,0,8,1073,0.285069,1
1581,06085511608,3238,215,137,3228,2039,347,38,25,9,...,0,52,36,30,0,18,0,78,0.362791,1


## Create share columns

Create columns which represent an estimate of a given language spoken as a share of the universe which is the population of people 5 years and over. 

In [184]:
def create_multiple_share_columns(dataframe, dictionary, total_population_column):
    for key, value in dictionary.items():
        dataframe[value] = np.where(dataframe[total_population_column] == 0,0,
                                    (dataframe[key] /
                                     dataframe[total_population_column]))

In [185]:
acs_df.columns

Index(['GEOID', 'total_population', 'total_pop_pov',
       'pop_200_pct_and_over_fpl', 'pop_5_years_over', 'pop_english_only',
       'spanish_english_vw', 'spanish_english_lvw', 'french_english_vw',
       'french_english_lvw', 'german_english_vw', 'german_english_lvw',
       'russian_english_vw', 'russian_english_lvw', 'other_indo_english_vw',
       'other_indo_english_lvw', 'korean_english_vw', 'korean_english_lvw',
       'chinese_english_vw', 'chinese_english_lvw', 'vietnamese_english_vw',
       'vietnamese_english_lvw', 'tagalog_english_vw', 'tagalog_english_lvw',
       'other_asian_english_vw', 'other_asian_english_lvw',
       'arabic_english_vw', 'arabic_english_lvw', 'other_unspec_english_vw',
       'other_unspec_english_lvw', 'pop_below_200_pct_fpl',
       'share_below_200_pct_fpl', 'low_income'],
      dtype='object')

In [186]:
pop_columns_list = [
    'spanish_english_lvw', 
    'french_english_lvw',
    'german_english_lvw',
    'russian_english_lvw',
    'other_indo_english_lvw',
    'korean_english_lvw',
    'chinese_english_lvw',
    'vietnamese_english_lvw',
    'tagalog_english_lvw',
    'other_asian_english_lvw',
    'arabic_english_lvw',
    'other_unspec_english_lvw'
]

In [187]:
#Initialize Dictionary with population column names and blank values
{key: 'pct_' + key for key in pop_columns_list}

{'spanish_english_lvw': 'pct_spanish_english_lvw',
 'french_english_lvw': 'pct_french_english_lvw',
 'german_english_lvw': 'pct_german_english_lvw',
 'russian_english_lvw': 'pct_russian_english_lvw',
 'other_indo_english_lvw': 'pct_other_indo_english_lvw',
 'korean_english_lvw': 'pct_korean_english_lvw',
 'chinese_english_lvw': 'pct_chinese_english_lvw',
 'vietnamese_english_lvw': 'pct_vietnamese_english_lvw',
 'tagalog_english_lvw': 'pct_tagalog_english_lvw',
 'other_asian_english_lvw': 'pct_other_asian_english_lvw',
 'arabic_english_lvw': 'pct_arabic_english_lvw',
 'other_unspec_english_lvw': 'pct_other_unspec_english_lvw'}

In [188]:
#Create a dictionary with population columns as keys and share columns as values
pop_share_cols_dictionary = {'spanish_english_lvw': 'pct_spanish_english_lvw',
 'french_english_lvw': 'pct_french_english_lvw',
 'german_english_lvw': 'pct_german_english_lvw',
 'russian_english_lvw': 'pct_russian_english_lvw',
 'other_indo_english_lvw': 'pct_other_indo_english_lvw',
 'korean_english_lvw': 'pct_korean_english_lvw',
 'chinese_english_lvw': 'pct_chinese_english_lvw',
 'vietnamese_english_lvw': 'pct_vietnamese_english_lvw',
 'tagalog_english_lvw': 'pct_tagalog_english_lvw',
 'other_asian_english_lvw': 'pct_other_asian_english_lvw',
 'arabic_english_lvw': 'pct_arabic_english_lvw',
 'other_unspec_english_lvw': 'pct_other_unspec_english_lvw'}

In [189]:
#Run share column creation function
create_multiple_share_columns(dataframe=acs_df,
                              dictionary=pop_share_cols_dictionary,
                              total_population_column='pop_5_years_over')

In [190]:
share_cols = list(pop_share_cols_dictionary.values())

In [191]:
acs_df[share_cols].sample(10)

,pct_spanish_english_lvw,pct_french_english_lvw,pct_german_english_lvw,pct_russian_english_lvw,pct_other_indo_english_lvw,pct_korean_english_lvw,pct_chinese_english_lvw,pct_vietnamese_english_lvw,pct_tagalog_english_lvw,pct_other_asian_english_lvw,pct_arabic_english_lvw,pct_other_unspec_english_lvw
807,0.086149,0.006194,0.000000,0.000000,0.000000,0.000000,0.009009,0.000000,0.022523,0.000000,0.000000,0.003660
484,0.043867,0.007853,0.001625,0.004333,0.014893,0.017872,0.041971,0.019767,0.000812,0.017601,0.000000,0.000000
185,0.245505,0.000000,0.000000,0.000000,0.003458,0.011296,0.018211,0.011987,0.008760,0.009682,0.000000,0.000000
126,0.077235,0.000000,0.000000,0.003134,0.009770,0.000000,0.028018,0.004793,0.008848,0.004240,0.003134,0.009954
785,0.069309,0.000000,0.001790,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002046,0.000000,0.000000
1010,0.044683,0.000000,0.000000,0.001966,0.054870,0.006077,0.198570,0.045398,0.032708,0.003217,0.000000,0.000000
1117,0.197880,0.000000,0.003298,0.000000,0.000000,0.000000,0.003062,0.000000,0.003769,0.000000,0.000000,0.000000
438,0.110072,0.000000,0.000000,0.004976,0.006170,0.000000,0.041003,0.044586,0.040207,0.015924,0.000000,0.000000
647,0.120876,0.000000,0.000000,0.000000,0.000000,0.000000,0.004317,0.000000,0.081098,0.012643,0.000000,0.000000
952,0.008161,0.000000,0.003536,0.018498,0.007345,0.000000,0.145811,0.000000,0.000000,0.012242,0.004625,0.000000


## Flag Non-English speaking population greater than 5% threshold

In [192]:
def flag_threshold(acs_df):
    if ((acs_df['pct_spanish_english_lvw'] >= .05) | 
        (acs_df['pct_french_english_lvw'] >= .05) |
        (acs_df['pct_german_english_lvw'] >= .05) |
        (acs_df['pct_russian_english_lvw'] >= .05) |
        (acs_df['pct_other_indo_english_lvw'] >= .05) |
        (acs_df['pct_korean_english_lvw'] >= .05) |
        (acs_df['pct_chinese_english_lvw'] >= .05) |
        (acs_df['pct_vietnamese_english_lvw'] >= .05) |
        (acs_df['pct_tagalog_english_lvw'] >= .05) |
        (acs_df['pct_other_asian_english_lvw'] >= .05) |
        (acs_df['pct_arabic_english_lvw'] >= .05) |
        (acs_df['pct_other_unspec_english_lvw'] >= .05)):
        return 1
    else:
        return 0

In [193]:
acs_df['over_5_pct_thres'] = acs_df.apply(flag_threshold,axis=1)

In [194]:
acs_df[acs_df['over_5_pct_thres'] == 1]

,GEOID,total_population,total_pop_pov,pop_200_pct_and_over_fpl,pop_5_years_over,pop_english_only,spanish_english_vw,spanish_english_lvw,french_english_vw,french_english_lvw,...,pct_russian_english_lvw,pct_other_indo_english_lvw,pct_korean_english_lvw,pct_chinese_english_lvw,pct_vietnamese_english_lvw,pct_tagalog_english_lvw,pct_other_asian_english_lvw,pct_arabic_english_lvw,pct_other_unspec_english_lvw,over_5_pct_thres
1,06097153801,10481,10450,8579,10035,6894,1618,1006,0,0,...,0.001993,0.000000,0.007474,0.000000,0.000000,0.000000,0.000000,0.000997,0.000000,1
2,06097150100,2456,2456,2195,2362,2034,100,180,10,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
3,06097150603,7481,7392,6255,7105,5272,903,675,0,0,...,0.001267,0.005630,0.001689,0.010837,0.000000,0.000000,0.000000,0.000000,0.000000,1
6,06097151301,4874,4874,3439,4432,2520,766,726,9,0,...,0.000000,0.027978,0.000000,0.003159,0.004738,0.000000,0.000000,0.000000,0.000000,1
8,06097153005,7024,6925,5557,6781,4681,794,938,0,0,...,0.000000,0.017402,0.000000,0.013125,0.000000,0.000000,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576,06085512310,4998,4998,3671,4689,2697,598,562,0,0,...,0.004479,0.016635,0.017274,0.026018,0.009810,0.000000,0.000000,0.002559,0.000000,1
1577,06085512314,6477,6344,4742,5744,3871,836,548,6,0,...,0.000000,0.007312,0.001915,0.003656,0.006442,0.001741,0.004004,0.012361,0.001567,1
1578,06085512313,3914,3859,3046,3638,2032,779,540,0,0,...,0.000000,0.000000,0.000000,0.003573,0.003848,0.003299,0.000550,0.000000,0.003024,1
1579,06085511302,4097,3954,3154,3937,2437,230,8,60,17,...,0.013208,0.000000,0.000000,0.078994,0.000000,0.000000,0.034798,0.000000,0.000000,1


In [195]:
{key: None for key in pop_share_cols_dictionary.values()}

{'pct_spanish_english_lvw': None,
 'pct_french_english_lvw': None,
 'pct_german_english_lvw': None,
 'pct_russian_english_lvw': None,
 'pct_other_indo_english_lvw': None,
 'pct_korean_english_lvw': None,
 'pct_chinese_english_lvw': None,
 'pct_vietnamese_english_lvw': None,
 'pct_tagalog_english_lvw': None,
 'pct_other_asian_english_lvw': None,
 'pct_arabic_english_lvw': None,
 'pct_other_unspec_english_lvw': None}

In [196]:
language_dict = {'pct_spanish_english_lvw': 'Spanish',
 'pct_french_english_lvw': 'French, Haitian, or Cajun',
 'pct_german_english_lvw': 'German or West Germanic',
 'pct_russian_english_lvw': 'Russian Polish of Slavic',
 'pct_other_indo_english_lvw': 'Other Indo-European',
 'pct_korean_english_lvw': 'Korean',
 'pct_chinese_english_lvw': 'Chinese',
 'pct_vietnamese_english_lvw': 'Vietnamese',
 'pct_tagalog_english_lvw': 'Tagalog',
 'pct_other_asian_english_lvw': 'Other Asian Pacific Island',
 'pct_arabic_english_lvw': 'Arabic',
 'pct_other_unspec_english_lvw': 'Other Unspecified'}

In [197]:
#Create a column to classify top non-english language spoken at home
acs_df['top_non_english_language'] = (acs_df[share_cols]
                                      .apply(lambda row: language_dict[row.idxmax()],axis=1))

In [198]:
share_cols

['pct_spanish_english_lvw',
 'pct_french_english_lvw',
 'pct_german_english_lvw',
 'pct_russian_english_lvw',
 'pct_other_indo_english_lvw',
 'pct_korean_english_lvw',
 'pct_chinese_english_lvw',
 'pct_vietnamese_english_lvw',
 'pct_tagalog_english_lvw',
 'pct_other_asian_english_lvw',
 'pct_arabic_english_lvw',
 'pct_other_unspec_english_lvw']

In [199]:
view_cols = share_cols
view_cols.append('top_non_english_language')

In [200]:
view_cols

['pct_spanish_english_lvw',
 'pct_french_english_lvw',
 'pct_german_english_lvw',
 'pct_russian_english_lvw',
 'pct_other_indo_english_lvw',
 'pct_korean_english_lvw',
 'pct_chinese_english_lvw',
 'pct_vietnamese_english_lvw',
 'pct_tagalog_english_lvw',
 'pct_other_asian_english_lvw',
 'pct_arabic_english_lvw',
 'pct_other_unspec_english_lvw',
 'top_non_english_language']

In [201]:
acs_df[view_cols]

,pct_spanish_english_lvw,pct_french_english_lvw,pct_german_english_lvw,pct_russian_english_lvw,pct_other_indo_english_lvw,pct_korean_english_lvw,pct_chinese_english_lvw,pct_vietnamese_english_lvw,pct_tagalog_english_lvw,pct_other_asian_english_lvw,pct_arabic_english_lvw,pct_other_unspec_english_lvw,top_non_english_language
0,0.040839,0.00000,0.000000,0.002483,0.000000,0.000000,0.004415,0.000000,0.000000,0.004691,0.000000,0.000000,Spanish
1,0.100249,0.00000,0.000000,0.001993,0.000000,0.007474,0.000000,0.000000,0.000000,0.000000,0.000997,0.000000,Spanish
2,0.076207,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Spanish
3,0.095004,0.00000,0.000000,0.001267,0.005630,0.001689,0.010837,0.000000,0.000000,0.000000,0.000000,0.000000,Spanish
4,0.040543,0.00191,0.001698,0.004458,0.001486,0.002759,0.003396,0.000000,0.000000,0.000000,0.000000,0.001486,Spanish
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583,0.000000,0.00000,0.000000,0.000000,0.003839,0.000000,0.026871,0.000000,0.007678,0.000000,0.000000,0.000000,Chinese
1584,0.018325,0.00000,0.000000,0.000000,0.000000,0.000000,0.012341,0.004488,0.000000,0.002618,0.000000,0.000000,Spanish
1585,0.148732,0.00000,0.002254,0.005070,0.010423,0.019437,0.000000,0.024225,0.000000,0.005634,0.000000,0.022254,Spanish
1586,0.032060,0.00000,0.000000,0.023647,0.011369,0.002501,0.037744,0.030468,0.002956,0.001137,0.002274,0.000000,Chinese


## Pull American community Service Tracts Geodata

### Query Census TIGER Tracts to get Bay Area Census Tract geography

#### [Census TIGERweb REST API Documentation](https://tigerweb.geo.census.gov/tigerwebmain/TIGERweb_restmapservice.html)

In [202]:
def pull_census_tracts_geodata(year=2020):
    """
    Pulls Census Tracts from TIGERweb REST API and returns Geopandas GeoDataframe. 
    Default year is 2020 which is the latest-available vintage for TIGER tracts. 
    
    Args:
    year (int): the TIGER vintage. 
        list of valid years: [2012,2015,2016,2017,2018,2019,2020]
    
    Author: Joshua Croff
    Source:
    [Census TIGERweb REST API Documentation](https://tigerweb.geo.census.gov/tigerwebmain/TIGERweb_restmapservice.html)
    """
    import pandas as pd
    valid_years = [2012,2015,2016,2017,2018,2019,2020]
    if year not in valid_years:
        print('Error- vintage not available. Please see docstring for valid years')
        return 
    
    state = '06'
    counties = "('001','013','041','055','075','081','085','095','097')"
    where_str = f"where=STATE='{state}'+AND+COUNTY+IN{counties}"
    query_args = [where_str, 'outFields=GEOID&f=geojson']
    url = '/'.join(['https://tigerweb.geo.census.gov',
              'arcgis',
              'rest',
              'services',
              f'Generalized_ACS{year}',
              'Tracts_Blocks',
              'MapServer',
              '3',
              'query?{}'.format('&'.join(query_args))])
    r = get_http_response(url)
    geog_json = r.json()
    geog_gdf = gp.GeoDataFrame.from_features(geog_json['features'],
                                        crs='EPSG:4326')
    return geog_gdf


## Join acs_df to tracts geodata

In [203]:
tracts_2018 = pull_census_tracts_geodata(year=2018)

In [204]:
acs_inc_lang_geo = pd.merge(tracts_2018,
                            acs_df,
                           on='GEOID',
                           how='inner')

In [206]:
reorder_cols = ['GEOID',
                'total_population',
                'total_pop_pov',
                'pop_200_pct_and_over_fpl',
                'pop_below_200_pct_fpl',
                'share_below_200_pct_fpl',
                'pop_5_years_over',
                'pop_english_only',
                'spanish_english_vw',
                'spanish_english_lvw',
                'french_english_vw',
                'french_english_lvw',
                'german_english_vw',
                'german_english_lvw',
                'russian_english_vw',
                'russian_english_lvw',
                'other_indo_english_vw',
                'other_indo_english_lvw',
                'korean_english_vw',
                'korean_english_lvw',
                'chinese_english_vw',
                'chinese_english_lvw',
                'vietnamese_english_vw',
                'vietnamese_english_lvw',
                'tagalog_english_vw',
                'tagalog_english_lvw',
                'other_asian_english_vw',
                'other_asian_english_lvw',
                'arabic_english_vw',
                'arabic_english_lvw',
                'other_unspec_english_vw',
                'other_unspec_english_lvw',
                'pct_spanish_english_lvw',
                'pct_french_english_lvw',
                'pct_german_english_lvw',
                'pct_russian_english_lvw',
                'pct_other_indo_english_lvw',
                'pct_korean_english_lvw',
                'pct_chinese_english_lvw',
                'pct_vietnamese_english_lvw',
                'pct_tagalog_english_lvw',
                'pct_other_asian_english_lvw',
                'pct_arabic_english_lvw',
                'pct_other_unspec_english_lvw',
                'top_non_english_language',
                'low_income',
                'over_5_pct_thres',
                'geometry']
acs_lang_geo = acs_inc_lang_geo[reorder_cols].copy()

In [207]:
acs_lang_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1582 entries, 0 to 1581
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   GEOID                         1582 non-null   object  
 1   total_population              1582 non-null   int64   
 2   total_pop_pov                 1582 non-null   int64   
 3   pop_200_pct_and_over_fpl      1582 non-null   int64   
 4   pop_below_200_pct_fpl         1582 non-null   int64   
 5   share_below_200_pct_fpl       1582 non-null   float64 
 6   pop_5_years_over              1582 non-null   int64   
 7   pop_english_only              1582 non-null   int64   
 8   spanish_english_vw            1582 non-null   int64   
 9   spanish_english_lvw           1582 non-null   int64   
 10  french_english_vw             1582 non-null   int64   
 11  french_english_lvw            1582 non-null   int64   
 12  german_english_vw             1582 non-n

In [208]:
acs_lang_geo.to_file('../data/low_income_over5_language_at_home_acs2014_2018.geojson',
                        driver='GeoJSON')

## Summarize data within study area buffer

Geographic processing performed in ArcGIS Pro software. Tracks within 7 mile buffer area exported to geojson
and read into notebook for summarization.

In [209]:
ls '../data'

B01003.xlsx
C16001.xlsx
LEP_Language_at_Home_880_Study_Area_Summary.csv
LEP_Language_at_Home_880_Study_Area_Tract.csv
acs_table_variables_income_language.csv
draft_regional_powerpoint.gdb/
draft_regional_powerpoint.tbx
low_income_over5_language_at_home_acs2014_2018.geojson
low_income_over5_language_at_home_acs2014_2018_study_area.geojson
otherData.gdb/
schema.ini


In [211]:
acs_lang_geo_study_area = gp.read_file('../data/low_income_over5_language_at_home_acs2014_2018_study_area.geojson',
                                      driver='GeoJSON')

In [212]:
acs_lang_geo_study_area.head(5)

,OBJECTID,GEOID,total_population,total_pop_pov,pop_200_pct_and_over_fpl,pop_below_200_pct_fpl,share_below_200_pct_fpl,pop_5_years_over,pop_english_only,spanish_english_vw,...,pct_tagalog_english_lvw,pct_other_asian_english_lvw,pct_arabic_english_lvw,pct_other_unspec_english_lvw,top_non_english_language,low_income,over_5_pct_thres,Shape_Length,Shape_Area,geometry
0,1,06001400200,2025,2018,1788,230,0.113974,1912,1536,83,...,0.000000,0.013598,0.000000,0.000000,Spanish,0,0,0.041276,0.000059,"POLYGON ((-122.25742 37.84310, -122.25792 37.8..."
1,2,06001400300,5000,4962,4436,526,0.106006,4704,3895,85,...,0.000000,0.005740,0.000000,0.049745,Other Unspecified,0,0,0.055914,0.000113,"POLYGON ((-122.26416 37.84000, -122.26563 37.8..."
2,3,06001400700,5116,5106,3186,1920,0.376028,4833,3572,281,...,0.000000,0.003724,0.024415,0.014898,Spanish,1,1,0.042183,0.000090,"POLYGON ((-122.27788 37.84751, -122.27798 37.8..."
3,4,06001400800,3792,3792,2318,1474,0.388713,3585,2425,105,...,0.008926,0.000000,0.017852,0.004742,Spanish,1,0,0.037540,0.000071,"POLYGON ((-122.28873 37.84996, -122.28787 37.8..."
4,5,06001403100,1668,1287,673,614,0.477078,1616,824,119,...,0.000000,0.016089,0.000000,0.003094,Chinese,1,1,0.024006,0.000035,"POLYGON ((-122.27956 37.80045, -122.27854 37.7..."


In [345]:
#Export study area data to csv
export_cols = [
 'GEOID',
 'total_population',
 'total_pop_pov',
 'pop_200_pct_and_over_fpl',
 'pop_below_200_pct_fpl',
 'share_below_200_pct_fpl',
 'pop_5_years_over',
 'pop_english_only',
 'spanish_english_vw',
 'spanish_english_lvw',
 'french_english_vw',
 'french_english_lvw',
 'german_english_vw',
 'german_english_lvw',
 'russian_english_vw',
 'russian_english_lvw',
 'other_indo_english_vw',
 'other_indo_english_lvw',
 'korean_english_vw',
 'korean_english_lvw',
 'chinese_english_vw',
 'chinese_english_lvw',
 'vietnamese_english_vw',
 'vietnamese_english_lvw',
 'tagalog_english_vw',
 'tagalog_english_lvw',
 'other_asian_english_vw',
 'other_asian_english_lvw',
 'arabic_english_vw',
 'arabic_english_lvw',
 'other_unspec_english_vw',
 'other_unspec_english_lvw',
 'pct_spanish_english_lvw',
 'pct_french_english_lvw',
 'pct_german_english_lvw',
 'pct_russian_english_lvw',
 'pct_other_indo_english_lvw',
 'pct_korean_english_lvw',
 'pct_chinese_english_lvw',
 'pct_vietnamese_english_lvw',
 'pct_tagalog_english_lvw',
 'pct_other_asian_english_lvw',
 'pct_arabic_english_lvw',
 'pct_other_unspec_english_lvw',
 'top_non_english_language',
 'low_income',
 'over_5_pct_thres']
acs_lang_geo_study_area[export_cols].to_csv('../data/low_income_over5_language_at_home_acs2014_2018_study_area.csv',index=False)

## Summarize universe populations for each ACS Survey, and populations of speakers of a language other than English at home that speak English very well. 

In [360]:
summary_cols = ['total_population',
             'total_pop_pov',
             'pop_below_200_pct_fpl',
             'pop_200_pct_and_over_fpl',
             'pop_5_years_over',
             'pop_english_only',
             'spanish_english_vw',
             'french_english_vw',
             'german_english_vw',
             'russian_english_vw',
             'other_indo_english_vw',
             'korean_english_vw',
             'chinese_english_vw',
             'vietnamese_english_vw',
             'tagalog_english_vw',
             'other_asian_english_vw',
             'arabic_english_vw',
             'other_unspec_english_vw']
pop_summaries = acs_lang_geo_study_area[summary_cols].aggregate('sum').to_frame().T
pop_summaries

,total_population,total_pop_pov,pop_below_200_pct_fpl,pop_200_pct_and_over_fpl,pop_5_years_over,pop_english_only,spanish_english_vw,french_english_vw,german_english_vw,russian_english_vw,other_indo_english_vw,korean_english_vw,chinese_english_vw,vietnamese_english_vw,tagalog_english_vw,other_asian_english_vw,arabic_english_vw,other_unspec_english_vw
0,1934278,1907214,467461,1439753,1813046,878446,204946,7651,5700,8160,90075,8541,71125,27135,52922,56246,7576,11561


## Aggregate low-income population and English Proficiency above theshold populations

In [361]:
summary_cols = [
 'spanish_english_lvw',
 'french_english_lvw',
 'german_english_lvw',
 'russian_english_lvw',
 'other_indo_english_lvw',
 'korean_english_lvw',
 'chinese_english_lvw',
 'vietnamese_english_lvw',
 'tagalog_english_lvw',
 'other_asian_english_lvw',
 'arabic_english_lvw',
 'other_unspec_english_lvw']
income_language_above_thresh_summary = (
    acs_lang_geo_study_area[(acs_lang_geo_study_area['over_5_pct_thres'] == 1)][summary_cols]
    .aggregate(sum)
    .to_frame()
    .T)

In [362]:
#Rename these columns to indicate that the summary is for language over threshold
income_language_above_thresh_summary.columns = [str(col) + '_above_thresh' for col in income_language_above_thresh_summary.columns]

In [363]:
income_language_above_thresh_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype
---  ------                                 --------------  -----
 0   spanish_english_lvw_above_thresh       1 non-null      int64
 1   french_english_lvw_above_thresh        1 non-null      int64
 2   german_english_lvw_above_thresh        1 non-null      int64
 3   russian_english_lvw_above_thresh       1 non-null      int64
 4   other_indo_english_lvw_above_thresh    1 non-null      int64
 5   korean_english_lvw_above_thresh        1 non-null      int64
 6   chinese_english_lvw_above_thresh       1 non-null      int64
 7   vietnamese_english_lvw_above_thresh    1 non-null      int64
 8   tagalog_english_lvw_above_thresh       1 non-null      int64
 9   other_asian_english_lvw_above_thresh   1 non-null      int64
 10  arabic_english_lvw_above_thresh        1 non-null      int64
 11  other_unspec_english_lvw_above_thres

## Aggregate low-income population and English Proficiency below theshold populations

In [364]:
summary_cols = [
 'spanish_english_lvw',
 'french_english_lvw',
 'german_english_lvw',
 'russian_english_lvw',
 'other_indo_english_lvw',
 'korean_english_lvw',
 'chinese_english_lvw',
 'vietnamese_english_lvw',
 'tagalog_english_lvw',
 'other_asian_english_lvw',
 'arabic_english_lvw',
 'other_unspec_english_lvw']
income_language_below_thresh_summary = (
    acs_lang_geo_study_area[(acs_lang_geo_study_area['over_5_pct_thres'] == 0)][summary_cols]
    .aggregate(sum)
    .to_frame()
    .T)

In [365]:
income_language_below_thresh_summary

,spanish_english_lvw,french_english_lvw,german_english_lvw,russian_english_lvw,other_indo_english_lvw,korean_english_lvw,chinese_english_lvw,vietnamese_english_lvw,tagalog_english_lvw,other_asian_english_lvw,arabic_english_lvw,other_unspec_english_lvw
0,9992,460,112,1583,5259,2150,10544,3596,2304,4466,915,1546


In [366]:
#Create a column which adds all languages above threshold together
income_language_below_thresh_summary['pop_other_languages_english_lvw_below_thresh'] = income_language_below_thresh_summary.sum(axis=1)


In [367]:
income_language_below_thresh_summary

,spanish_english_lvw,french_english_lvw,german_english_lvw,russian_english_lvw,other_indo_english_lvw,korean_english_lvw,chinese_english_lvw,vietnamese_english_lvw,tagalog_english_lvw,other_asian_english_lvw,arabic_english_lvw,other_unspec_english_lvw,pop_other_languages_english_lvw_below_thresh
0,9992,460,112,1583,5259,2150,10544,3596,2304,4466,915,1546,42927


## Concatenate all dataframes together

In [368]:
summary_df = pd.concat([pop_summaries,
           income_language_above_thresh_summary,
          income_language_below_thresh_summary['pop_other_languages_english_lvw_below_thresh']],axis=1)

In [369]:
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 31 columns):
 #   Column                                        Non-Null Count  Dtype
---  ------                                        --------------  -----
 0   total_population                              1 non-null      int64
 1   total_pop_pov                                 1 non-null      int64
 2   pop_below_200_pct_fpl                         1 non-null      int64
 3   pop_200_pct_and_over_fpl                      1 non-null      int64
 4   pop_5_years_over                              1 non-null      int64
 5   pop_english_only                              1 non-null      int64
 6   spanish_english_vw                            1 non-null      int64
 7   french_english_vw                             1 non-null      int64
 8   german_english_vw                             1 non-null      int64
 9   russian_english_vw                            1 non-null      int64
 10  other_indo_english

## Create summary share columns

In [370]:
summary_df.rename(columns={'pop_below_200_pct_fpl':'pop_low_income'},inplace=True)

In [371]:
summary_df['low_income_pct'] = summary_df['pop_low_income'] / summary_df['total_pop_pov']

In [372]:
summary_df

,total_population,total_pop_pov,pop_low_income,pop_200_pct_and_over_fpl,pop_5_years_over,pop_english_only,spanish_english_vw,french_english_vw,german_english_vw,russian_english_vw,...,other_indo_english_lvw_above_thresh,korean_english_lvw_above_thresh,chinese_english_lvw_above_thresh,vietnamese_english_lvw_above_thresh,tagalog_english_lvw_above_thresh,other_asian_english_lvw_above_thresh,arabic_english_lvw_above_thresh,other_unspec_english_lvw_above_thresh,pop_other_languages_english_lvw_below_thresh,low_income_pct
0,1934278,1907214,467461,1439753,1813046,878446,204946,7651,5700,8160,...,23143,5984,88005,38111,23315,17497,3121,5827,42927,0.245101


## Create share columns from language populations

In [373]:
summary_df.columns

Index(['total_population', 'total_pop_pov', 'pop_low_income',
       'pop_200_pct_and_over_fpl', 'pop_5_years_over', 'pop_english_only',
       'spanish_english_vw', 'french_english_vw', 'german_english_vw',
       'russian_english_vw', 'other_indo_english_vw', 'korean_english_vw',
       'chinese_english_vw', 'vietnamese_english_vw', 'tagalog_english_vw',
       'other_asian_english_vw', 'arabic_english_vw',
       'other_unspec_english_vw', 'spanish_english_lvw_above_thresh',
       'french_english_lvw_above_thresh', 'german_english_lvw_above_thresh',
       'russian_english_lvw_above_thresh',
       'other_indo_english_lvw_above_thresh',
       'korean_english_lvw_above_thresh', 'chinese_english_lvw_above_thresh',
       'vietnamese_english_lvw_above_thresh',
       'tagalog_english_lvw_above_thresh',
       'other_asian_english_lvw_above_thresh',
       'arabic_english_lvw_above_thresh',
       'other_unspec_english_lvw_above_thresh',
       'pop_other_languages_english_lvw_below_t

In [328]:
lang_cols = ['pop_english_only_li','pop_other_languages_english_vw_li', 'spanish_english_lvw_li_a_thresh',
       'french_english_lvw_li_a_thresh', 'german_english_lvw_li_a_thresh',
       'russian_english_lvw_li_a_thresh', 'other_indo_english_lvw_li_a_thresh',
       'korean_english_lvw_li_a_thresh', 'chinese_english_lvw_li_a_thresh',
       'vietnamese_english_lvw_li_a_thresh', 'tagalog_english_lvw_li_a_thresh',
       'other_asian_english_lvw_li_a_thresh', 'arabic_english_lvw_li_a_thresh',
       'other_unspec_english_lvw_li_a_thresh',
       'pop_other_languages_english_lvw_li_b_thresh']

In [329]:
#Initialize Dictionary with population column names and blank values
{key: 'pct_' + key for key in lang_cols}

{'pop_english_only_li': 'pct_pop_english_only_li',
 'pop_other_languages_english_vw_li': 'pct_pop_other_languages_english_vw_li',
 'spanish_english_lvw_li_a_thresh': 'pct_spanish_english_lvw_li_a_thresh',
 'french_english_lvw_li_a_thresh': 'pct_french_english_lvw_li_a_thresh',
 'german_english_lvw_li_a_thresh': 'pct_german_english_lvw_li_a_thresh',
 'russian_english_lvw_li_a_thresh': 'pct_russian_english_lvw_li_a_thresh',
 'other_indo_english_lvw_li_a_thresh': 'pct_other_indo_english_lvw_li_a_thresh',
 'korean_english_lvw_li_a_thresh': 'pct_korean_english_lvw_li_a_thresh',
 'chinese_english_lvw_li_a_thresh': 'pct_chinese_english_lvw_li_a_thresh',
 'vietnamese_english_lvw_li_a_thresh': 'pct_vietnamese_english_lvw_li_a_thresh',
 'tagalog_english_lvw_li_a_thresh': 'pct_tagalog_english_lvw_li_a_thresh',
 'other_asian_english_lvw_li_a_thresh': 'pct_other_asian_english_lvw_li_a_thresh',
 'arabic_english_lvw_li_a_thresh': 'pct_arabic_english_lvw_li_a_thresh',
 'other_unspec_english_lvw_li_a_t

In [330]:
language_dict = {'pop_english_only_li': 'pct_english_only_li',
 'pop_other_languages_english_vw_li': 'pct_other_languages_english_vw_li',
 'spanish_english_lvw_li_a_thresh': 'pct_spanish_english_lvw_li_a_thresh',
 'french_english_lvw_li_a_thresh': 'pct_french_english_lvw_li_a_thresh',
 'german_english_lvw_li_a_thresh': 'pct_german_english_lvw_li_a_thresh',
 'russian_english_lvw_li_a_thresh': 'pct_russian_english_lvw_li_a_thresh',
 'other_indo_english_lvw_li_a_thresh': 'pct_other_indo_english_lvw_li_a_thresh',
 'korean_english_lvw_li_a_thresh': 'pct_korean_english_lvw_li_a_thresh',
 'chinese_english_lvw_li_a_thresh': 'pct_chinese_english_lvw_li_a_thresh',
 'vietnamese_english_lvw_li_a_thresh': 'pct_vietnamese_english_lvw_li_a_thresh',
 'tagalog_english_lvw_li_a_thresh': 'pct_tagalog_english_lvw_li_a_thresh',
 'other_asian_english_lvw_li_a_thresh': 'pct_other_asian_english_lvw_li_a_thresh',
 'arabic_english_lvw_li_a_thresh': 'pct_arabic_english_lvw_li_a_thresh',
 'other_unspec_english_lvw_li_a_thresh': 'pct_other_unspec_english_lvw_li_a_thresh',
 'pop_other_languages_english_lvw_li_b_thresh': 'pct_other_languages_english_lvw_li_b_thresh'}

In [331]:
create_multiple_share_columns(dataframe=summary_df,
                              dictionary=language_dict,
                              total_population_column='pop_5_years_over')

In [332]:
summary_df

,total_population,total_pop_pov,pop_5_years_over,pop_low_income,pop_english_only_li,pop_other_languages_english_vw_li,spanish_english_lvw_li_a_thresh,french_english_lvw_li_a_thresh,german_english_lvw_li_a_thresh,russian_english_lvw_li_a_thresh,...,pct_russian_english_lvw_li_a_thresh,pct_other_indo_english_lvw_li_a_thresh,pct_korean_english_lvw_li_a_thresh,pct_chinese_english_lvw_li_a_thresh,pct_vietnamese_english_lvw_li_a_thresh,pct_tagalog_english_lvw_li_a_thresh,pct_other_asian_english_lvw_li_a_thresh,pct_arabic_english_lvw_li_a_thresh,pct_other_unspec_english_lvw_li_a_thresh,pct_other_languages_english_lvw_li_b_thresh
0,1934278,1907214,1813046,269892,283584,166663,87962,262,160,773,...,0.000426,0.002774,0.000552,0.015138,0.008146,0.003412,0.002989,0.00119,0.00244,0.004173


In [334]:
reorder_cols = ['total_population',
 'total_pop_pov',
 'pop_5_years_over',
 'pop_low_income',
 'low_income_pct',
 'pop_english_only_li',
 'pct_english_only_li',
 'pop_other_languages_english_vw_li',
 'pct_other_languages_english_vw_li',
 'spanish_english_lvw_li_a_thresh',
 'pct_spanish_english_lvw_li_a_thresh',
 'french_english_lvw_li_a_thresh',
 'pct_french_english_lvw_li_a_thresh',
 'german_english_lvw_li_a_thresh',
 'pct_german_english_lvw_li_a_thresh',
 'russian_english_lvw_li_a_thresh',
 'pct_russian_english_lvw_li_a_thresh',
 'other_indo_english_lvw_li_a_thresh',
 'pct_other_indo_english_lvw_li_a_thresh',
 'korean_english_lvw_li_a_thresh',
 'pct_korean_english_lvw_li_a_thresh',
 'chinese_english_lvw_li_a_thresh',
 'pct_chinese_english_lvw_li_a_thresh',
 'vietnamese_english_lvw_li_a_thresh',
 'pct_vietnamese_english_lvw_li_a_thresh',
 'tagalog_english_lvw_li_a_thresh',
 'pct_tagalog_english_lvw_li_a_thresh',
 'other_asian_english_lvw_li_a_thresh',
 'pct_other_asian_english_lvw_li_a_thresh',
 'arabic_english_lvw_li_a_thresh',
 'pct_arabic_english_lvw_li_a_thresh',
 'other_unspec_english_lvw_li_a_thresh',
 'pct_other_unspec_english_lvw_li_a_thresh',
 'pop_other_languages_english_lvw_li_b_thresh',
 'pct_other_languages_english_lvw_li_b_thresh']
summary_df = summary_df[reorder_cols]

In [337]:
summary_df_transpose = summary_df.T

In [339]:
summary_df_transpose.rename(columns={0:'Summary_Statistics'},inplace=True)

In [340]:
summary_df_transpose

,Summary_Statistics
total_population,1.934278e+06
total_pop_pov,1.907214e+06
pop_5_years_over,1.813046e+06
pop_low_income,2.698920e+05
low_income_pct,1.415111e-01
pop_english_only_li,2.835840e+05
pct_english_only_li,1.564130e-01
pop_other_languages_english_vw_li,1.666630e+05
pct_other_languages_english_vw_li,9.192431e-02
spanish_english_lvw_li_a_thresh,8.796200e+04


In [341]:
summary_df_transpose.to_csv('../data/Language_at_Home_Low_Income_Summary_Statistics.csv')